# TP2 : 
## Algorithmes itératifs pour les processus de décision Markovien

Dans ce TP2 sur les algorithmes itératifs pour les processus de décision Markovien, l'objectif est d'appliquer l'algorithme d'itération de la valeur pour déterminer la politique optimale dans un environnement de grille (gridworld). Ce problème met en évidence l'application des principes des processus de décision Markoviens (MDP) à un environnement simple mais illustratif.

## Définitions des fonctions :

- Définition de la Fonction de Mouvement (move):
 Cette fonction Retourne les coordonnées de l'état après l'action.

- Validation des Mouvements:
 La fonction is_valid_move vérifie si un mouvement proposé est valide, c'est-à-dire qu'il ne sort pas des limites de la grille et qu'il ne passe pas à travers un mur.

- Actions Possibles:
 possible_actions retourne une liste des actions valides à partir d'une position donnée, en utilisant is_valid_move pour filtrer les actions non valides.

- Rotation des Actions de 90 Degrés:
 action_90degree est une fonction destinée à identifier les actions perpendiculaires à une action donnée.

- get_state_index :
 Fonction pour obtenir l'indice linéaire d'un état.

In [5]:
import numpy as np
        

def move (state, action):
    x, y = state
    if action == (1,0):
        next_state = (x+1, y)
    elif action == (-1,0):
        next_state = (x-1, y)
    elif action == (0,1):
        next_state = (x, y+1)
    elif action == (0,-1):
        next_state = (x, y-1)
    return next_state

def is_valid_move(position, action):
    new_position = move(position, action)
    x, y = new_position
    if 0 <= x < n_rows and 0 <= y < n_cols and new_position != wall:
        return True
    return False

def possible_actions(position):
    valid_actions = []
    if position != wall:
        for action in actions:
            if is_valid_move(position, action):
                valid_actions.append(action)
    return valid_actions

def action_90degree (action,actions_possible) :
    actions= []
    for i in actions_possible :
        if i != action:
            if i+action != (0,0):
                actions.append(i)
    return actions

def get_state_index(x, y):
    return x * n_cols + y

### Initialisation 
- Initialisation de la Table des Récompenses:
 On commence par initialiser une grille de récompenses avec des zéros et on définit des récompenses spécifiques pour certains états, comme un état objectif (goal) avec une récompense de 1 et un état pénalisant (bad) avec une récompense de -1.



In [6]:

n_rows, n_cols = 3, 4 
actions = [(1 ,0), (-1,0), (0,-1), (0,1)]
goal = (0 , 3)  # Coordonnées de l'état objectif
bad = (1 , 3)   # Coordonnées de l'état pénalisant
wall = (1, 1)  # Position du mur (en indexation à partir de 0)

n_states = n_rows * n_cols
gamma = 0.9  # Facteur d'escompte
threshold = 0.01  # Seuil de convergence
V = np.zeros(n_states)
V[get_state_index(0,3)]=1
V[get_state_index(1,3)]=-1

reward = np.zeros((n_rows, n_cols))
reward[goal] = 1
reward[bad] = -1

# Parcourir la grille et afficher les actions possibles pour chaque case
for x in range(n_rows):
    for y in range(n_cols):
        position = (x, y)
        if position == goal:
            exp = "<== goal "
        elif position == bad:
            exp = "<== bad "
        elif position == wall:
            exp = "<== wall "
        else:
            exp = ""
        print(f"Actions possibles depuis la position  {position} : {possible_actions(position)} {exp}")


# Affichage de la table des récompenses initiale
print("Table des récompenses initial:")
print(V.reshape (n_rows, n_cols))

NameError: name 'n_rows' is not defined

- ### Itération de la Valeur:
 Au cœur de l'algorithme, on itère sur chaque état de la grille, en excluant le mur, l'état objectif et l'état pénalisant de la mise à jour puisque leur valeur est déjà définie. Pour chaque état restant, on calcule les récompenses attendues pour chaque action possible en tenant compte à la fois de l'action principale et des déviations possibles.

Calcul de la Somme des Récompenses: Pour chaque action, on calcule la somme des récompenses attendues, en ajustant la probabilité en fonction du nombre d'actions de déviation possibles (0.1 pour deux déviations possibles, 0.2 pour une seule). Cette somme prend en compte la récompense immédiate de l'état, la valeur future escomptée des états accessibles à partir de l'action actuelle et des actions de déviation.
Convergence: L'algorithme continue d'itérer jusqu'à ce que le changement maximal dans les valeurs des états entre deux itérations consécutives tombe en dessous d'un seuil de convergence prédéfini (threshold == 0.01).

In [ ]:
while True:
    delta = 0
    for x in range(n_rows):
        for y in range(n_cols):
            if (x, y) == wall or (x,y) == goal  or (x,y) == bad:
                continue  # Pas de calcul  pour ces cases
            v = V[get_state_index(x, y)]
            Vs = []
            pa=possible_actions((x, y))
            for action in pa:
                for other_action in action_90degree(action,pa) :
                    if len(action_90degree(action,pa)) == 1 :
                        proba=0.2
                    else :
                        proba=0.1
                    sum_derivations= proba * V[get_state_index(*move((x, y), other_action))]
                sum_action = reward[x, y] + gamma * 0.8 * V[get_state_index(*move((x, y), action))]
                sum_rewards = sum_action + sum_derivations
                Vs.append(sum_rewards)
            V[get_state_index(x, y)] = max(Vs)
            delta = max(delta, abs(v - V[get_state_index(x, y)]))
    if delta < threshold:
        break

print("Fonction de valeur optimale :")
print (V.reshape(n_rows, n_cols ))

Fonction de valeur optimale :
[[ 0.59675907  0.68715851  0.78871585  1.        ]
 [ 0.51881065  0.          0.46787541 -1.        ]
 [ 0.45137749  0.39326998  0.34145044  0.04584432]]


Politique optimale π* : (normalement)
[[(0, 1) (0, 1) (0, 1) None]
 [(0, 1) None (1, 0) None]
 [(0, 1) (0, 1) (1, 0) (0, -1)]]


In [ ]:
import numpy as np

# Initialiser la politique avec None pour indiquer qu'aucune action n'est encore choisie
π_star = np.full((n_rows, n_cols), None)

# Itération sur chaque état pour déterminer l'action qui maximise la somme des récompenses
for x in range(n_rows):
    for y in range(n_cols):
        # Exclure le goal et le bad
        if (x, y) == goal or (x, y) == bad:
            continue

        best_value = float('-inf')
        best_action = None
        pa = possible_actions((x, y))
        for action in pa:
            action_value = reward[x, y]
            transition_states = action_90degree(action, pa)  # Les états possibles après avoir pris 'action'
            action_prob = 0.8  # Probabilité de prendre l'action prévue
            deviation_prob = (1 - action_prob) / len(transition_states) if transition_states else 0
            
            # Calculer la valeur attendue de prendre l'action
            for other_action in transition_states:
                next_state = move((x, y), other_action)
                action_value += deviation_prob * V[get_state_index(*next_state)]
            
            # Ajouter la valeur attendue si l'action principale est exécutée avec succès
            next_state = move((x, y), action)
            action_value += action_prob * V[get_state_index(*next_state)]
            
            # Si la valeur calculée pour cette action est meilleure, la retenir
            if action_value > best_value:
                best_value = action_value
                best_action = action
        
        π_star[x, y] = best_action  # Mettre à jour la politique optimale avec la meilleure action

# Mettre None pour le goal et le bad dans la politique optimale
π_star[goal[0], goal[1]] = "Goal"
π_star[bad[0], bad[1]] = "Bad"
π_star[ wall[0], wall[1]] = "Wall"

print("Politique optimale π* :")
print(π_star)


Politique optimale π* :
[[(0, 1) (0, 1) (0, 1) 'Goal']
 [(-1, 0) 'Wall' (-1, 0) 'Bad']
 [(-1, 0) (0, -1) (-1, 0) (0, -1)]]
